In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import akshare as ak
import bt

import matplotlib as mpl

mpl.rcParams["font.sans-serif"] = ["SimHei"]
mpl.rcParams["axes.unicode_minus"] = False

## 均线交叉结合通道突破择时研究


In [6]:
hs300 = ak.stock_zh_index_daily(symbol="sh00300")
hs300.display()
df_data = hs300[(hs300["date"] > pd.to_datetime("2005-01-01")) & (hs300["date"] < pd.to_datetime("2024-10-18"))]
df_data["date"] = pd.to_datetime(df_data["date"])
df_data = df_data.set_index(["date"], drop=True).astype("float")
close_ = df_data[["close"]]

KeyError: 'date'

### 均线交叉结合通道突破和动态止损
在快速均线（`FastWindow`个交易日的均线）上穿慢速均线（`SlowWindow`个交易日的均线）时（金叉），认为价格有上涨趋势。快速均线下穿慢速均线时（死叉），认为价格有下跌趋势。出现均线交叉信号后的ChLen个交易日内，若价格向上突破阻力位时发出买入信号，价格向下突破支撑位时发出卖出信号，反手出场。出现均线交叉信号的ChLen个交易日之后此信号失效。 持仓时设置跟踪止损触发价为`TrailWindow`个交易日的日内最低价的最低者。价格跌破该触发价时进行止损，暂时出场。在出场后，设置阻力线为最近`ReEntryChLen`个交易日内最高价的高点。若在出场后的`ReEntryWindow`个交易日内价格向上突破此阻力线，判定原趋势继续，再度进场。出场`ReEntryWindow`个交易日之后此信号失效。

本策略主要涉及快速均线、慢速均线、通道突破、跟踪止损。快速均线上穿慢速均线（金叉）意味着短期上涨趋势，快速均线下穿慢速均线（死叉）意味着短期下跌趋势。在识别趋势后设置进出场突破通道，价格向上突破阻力线时发出买入信号，价格向下突破支撑位时发出卖出信号。持仓时采用移动窗口，将近期日内最低价的最低者作为跟踪止损触发价，价格跌破此支撑位时进行跟踪止损。出场后可能出现趋势继续，需要重新进场。将出场后的日内最高价的最高者作为再进场买入突破价。当价格向上突破该买入触发价时再次进场。

In [ ]:
ma09 = close_.rolling(window=9).mean()
ma18 = close_.rolling(window=18).mean()

GoldenCross = (ma09 > ma18) & (ma09.shift(1) <= ma18.shift(1))
DeathCross = (ma09 < ma18) & (ma09.shift(1) >= ma18.shift(1))

df_sig = pd.DataFrame(np.nan, index=close_.index, columns=close_.columns)
df_sig[GoldenCross] = 1
df_sig[DeathCross] = 0
df_sig = df_sig.fillna(method='ffill').shift(1)

# 沪深300
s0 = bt.Strategy('沪深300指数净值', [bt.algos.RunOnce(),
                       bt.algos.SelectAll(),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
t0 = bt.Backtest(s0, close_)

# 均线策略
s1 = bt.Strategy('简单均线交叉策略净值(双边各0.3%)', [bt.algos.RunDaily(),
                       bt.algos.SelectWhere(df_sig>=1),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
t1 = bt.Backtest(s1, close_, commissions=lambda q, p: abs(q)*p*3/1000)

res = bt.run(t0, t1)
res.plot(title="简单均线交叉策略")
res.display()

In [ ]:
# 参数
FastWindow = 9   
ExtraPct = 3.0 / 100  
SlowWindow = FastWindow * 2  
ChLen = FastWindow + 3  
TrailWindow = FastWindow - 1  
ReEntryWindow = FastWindow * 2 -3  
ReEntryChLen = FastWindow + 1
# 均线交叉信号
df_data['FastMA'] = df_data['close'].rolling(window=FastWindow).mean()
df_data['SlowMA'] = df_data['close'].rolling(window=SlowWindow).mean()

df_data['GoldenCross'] = (df_data['FastMA'] > df_data['SlowMA']) & (df_data['FastMA'].shift(1) <= df_data['SlowMA'].shift(1))
df_data['DeathCross'] = (df_data['FastMA'] < df_data['SlowMA']) & (df_data['FastMA'].shift(1) >= df_data['SlowMA'].shift(1))

# 通道突破信号
df_data['marker'] = np.nan

GoldenCross_indices = df_data.index[df_data['GoldenCross']]
for idx in GoldenCross_indices:
    start_idx = df_data.index.get_loc(idx)
    end_idx = start_idx + ChLen
    window = df_data.iloc[start_idx:end_idx]

    if window.empty :
        continue

    index_tmp = window.index[window['close'] > df_data.loc[idx, 'close'] * (1 + ExtraPct)]
    if 0 == len(index_tmp):
        continue

    df_data.at[index_tmp[0], 'marker'] = True

DeathCross_indices = df_data.index[df_data['DeathCross']]
for idx in DeathCross_indices:
    start_idx = df_data.index.get_loc(idx)
    end_idx = start_idx + ChLen
    window = df_data.iloc[start_idx:end_idx]

    if window.empty :
        continue

    index_tmp = window.index[window['close'] < df_data.loc[idx, 'close'] * (1 - ExtraPct)]
    if 0 == len(index_tmp):
        continue

    df_data.at[index_tmp[0], 'marker'] = False

# 跟踪止损信号
df_data['marker1'] = np.nan

df_data['sig'] = df_data['marker'].fillna(method='ffill')
df_data['segment'] = (df_data['sig'] != df_data['sig'].shift()).cumsum()
segments = [group for _, group in df_data.groupby('segment')]

for i, segment in enumerate(segments):
    segment['marker1'] = segment['marker']
    if(list(segment['marker'])[0] == True):
        count = 0
        prev_idx = None
        prev_state = True
        for index, row in segment.iterrows():
            count+=1
            if count <= TrailWindow+1:
                continue
            if True == prev_state:
                end_idx = segment.index.get_loc(index) -1
                start_idx = end_idx - TrailWindow
                if row['close'] < segment.iloc[start_idx:end_idx]['low'].min():
                    segment.loc[index, 'marker1'] = False
                    prev_state = False
                    prev_idx = count
            else:
                if count - prev_idx > ReEntryWindow:
                    continue
                end_idx = segment.index.get_loc(index) -1
                start_idx = end_idx - ReEntryChLen
                if row['close'] > segment.iloc[start_idx:end_idx]['high'].max():
                    segment.loc[index, 'marker1'] = True
                    prev_state = True
                    prev_idx = count
    df_data.loc[segment.index, 'marker1'] = segment['marker1']

# 回测
df_sig = pd.DataFrame(np.nan, index=close_.index, columns=close_.columns)
df_sig['close'] = df_data['FastMA'] > df_data['SlowMA']

df_sig1 = pd.DataFrame(np.nan, index=close_.index, columns=close_.columns)
df_sig1['close'] = df_data['marker']
df_sig1 = df_sig1.fillna(method='ffill').shift(1)

df_sig2 = pd.DataFrame(np.nan, index=close_.index, columns=close_.columns)
df_sig2['close'] = df_data['marker1']
df_sig2 = df_sig2.fillna(method='ffill').shift(1)

# 沪深300
s0 = bt.Strategy('沪深300指数净值', [bt.algos.RunOnce(),
                       bt.algos.SelectAll(),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
t0 = bt.Backtest(s0, close_)

s1 = bt.Strategy('简单均线交叉策略净值(双边各0.3%)', [bt.algos.RunDaily(),
                       bt.algos.SelectWhere(df_sig),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
t1 = bt.Backtest(s1, close_, commissions=lambda q, p: abs(q)*p*3/1000)

s2 = bt.Strategy('均线交叉+通道突破(双边各0.3%)', [bt.algos.RunDaily(),
                       bt.algos.SelectWhere(df_sig1),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
t2 = bt.Backtest(s2, close_, commissions=lambda q, p: abs(q)*p*3/1000)

s3 = bt.Strategy('均线交叉+通道突破+跟踪止损(双边各0.3%)', [bt.algos.RunDaily(),
                       bt.algos.SelectWhere(df_sig2),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
t3 = bt.Backtest(s3, close_, commissions=lambda q, p: abs(q)*p*3/1000)

res = bt.run(t0, t1, t2, t3)
res.plot(title="各策略净值对比")
res.display()